# Imports and env

In [13]:
import re, random, os
import json, requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
from nltk.stem import PorterStemmer
# from nltk.stem import LancasterStemmer
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

import seaborn as sns
import enchant
from pathlib import Path
from datasets import list_datasets, load_dataset
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to C:\Users\Michael
[nltk_data]     Minut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Michael
[nltk_data]     Minut\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Michael
[nltk_data]     Minut\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Michael
[nltk_data]     Minut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
!python -m spacy download ro_core_news_sm

[+] Download and installation successful
You can now load the package via spacy.load('ro_core_news_sm')


You should consider upgrading via the 'E:\coding\repos\DLNLPProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
nlp = spacy.load("ro_core_news_sm")

In [14]:
dict_en = enchant.Dict("en_US")

In [19]:
dict_en.check("duh")

True

# Sanitization Class

In [30]:
# ======================================================================================================================
# Class - PreprocessExpert
# ======================================================================================================================
class PreprocessExpert:

    RO_STOP_WORDS = None
    EN_ACRONYMS = None
    PORTER_STEM = None
    WORDNET_LEMMATIZER = None

    @staticmethod
    def load_stop_words_ro():
        # From: https://www.kaggle.com/code/mpwolke/romanian-stop-words-w2v/data?select=romanian.txt
        with open(r'./data_files/romanian_stop_words.txt', 'r') as f:
            ro_sw = f.readlines()
            ro_sw = set(w.strip() for w in ro_sw)
            PreprocessExpert.RO_STOP_WORDS = list(frozenset(ro_sw))
    
    @staticmethod
    def remove_stop_words_ro(doc):
        if PreprocessExpert.RO_STOP_WORDS is None:
            PreprocessExpert.load_stop_words_ro()
        new_doc = ' '.join([w for w in doc.split() if w not in PreprocessExpert.RO_STOP_WORDS])
        return new_doc

    @staticmethod
    def remove_stop_words_en(doc):
        # 
        new_doc = ' '.join([w for w in doc.split() if w not in stopwords.words()])
        return new_doc

    @staticmethod
    def download_acronyms():
        # https://www.netlingo.com/acronyms.php
        # stack python-module-to-remove-internet-jargon-slang-acronym
        # TODO? ro: http://abrevierile.ro/colectii
        resp = requests.get("http://www.netlingo.com/acronyms.php")
        soup = BeautifulSoup(resp.text, "html.parser")
        slangdict = {}
        key = ""
        value = ""
        for div in soup.findAll('div', attrs={'class':'list_box3'}):
            for li in div.findAll('li'):
                for a in li.findAll('a'):
                    key = a.text
                    value = li.text.split(key)[1]
                    slangdict[key.lower()]=value

        with open('data_files/acronym_en.json', 'w') as f:
            json.dump(slangdict, f, indent=2)

    @staticmethod
    def load_acronyms():
        # Loads acronyms data
        if not os.path.exists('data_files/acronym_en.json'):
            PreprocessExpert.download_acronyms()
        with open('data_files/acronym_en.json', 'r') as f:
            PreprocessExpert.EN_ACRONYMS = json.load(f)
        # print(PreprocessExpert.EN_ACRONYMS)

    @staticmethod
    def remove_acronyms(doc):
        # replace...
        # if PreprocessExpert.EN_ACRONYMS is None:
        #     PreprocessExpert.load_acronyms()
        new_doc = ' '.join([w for w in doc.split() if w not in PreprocessExpert.EN_ACRONYMS.keys()])
        return new_doc

    @staticmethod
    def remove_email_addr(doc):
        # Removes email strings from doc
        new_doc = re.sub(r'[A-Za-z0-9]{1,}@[A-Za-z]{1,}\.[A-Za-z0-9]{1,}', "", doc)
        return new_doc

    @staticmethod
    def stemm_en(doc):
        # Stemming en words of doc
        new_doc = []
        our_words = doc.split()
        if PreprocessExpert.PORTER_STEM is None:
            PreprocessExpert.PORTER_STEM = PorterStemmer()
        for w in our_words:
            # if word is of known en
            if PreprocessExpert.is_english_word(w):
                new_doc.append(PreprocessExpert.PORTER_STEM.stem(w))
            else:
                new_doc.append(w)
        return ' '.join(new_doc)

    @staticmethod
    def lemma_en(doc):
        # Lemmatization of en words in doc
        new_doc = []
        our_words = doc.split()
        if PreprocessExpert.WORDNET_LEMMATIZER is None:
            PreprocessExpert.WORDNET_LEMMATIZER = WordNetLemmatizer()
        for w in our_words:
            # if word is of known en
            if PreprocessExpert.is_english_word(w):
                new_doc.append(PreprocessExpert.WORDNET_LEMMATIZER.lemmatize(w))
            else:
                new_doc.append(w)
        return ' '.join(new_doc)
    
    @staticmethod
    def replace_ro_diacritics(doc):
        diacritics_counterparts = {"Ă":"A",
                        "ă":"a",
                        "Â":"A",
                        "â":"a",
                        "Î": "I",
                        "î": "i",
                        "Ș":"S",
                        "ș":"s",
                        "Ț":"T",
                        "ț":"t",
                        "Ş":"S",
                        "ş":"s"}
        for index, letter in enumerate(doc):
            if letter in diacritics_counterparts:
                doc[index] = diacritics_counterparts[letter]
        return doc
    
    @staticmethod
    def is_english_word(word):
        return dict_en.check(word)
    
    @staticmethod
    def remove_ulrs(doc):
        return re.sub(r'https?://\S+', "", doc)
    
    @staticmethod
    def remove_hashtags(doc):
        return re.sub(r'(@[A-Za-z0-9]+)', "", doc)
    
    @staticmethod
    def get_lemma_ro(text):
        try:
            doc = nlp(text)
            empty_list = []
            for token in doc:
                empty_list.append(token.lemma_)

            final_string = ' '.join(map(str,empty_list))
            return final_string
        except:
            return text
        
    @staticmethod
    def nltk_stopwords_ro(phrases):
        stopwords = nltk.corpus.stopwords.words('romanian')
        removed_stopwords = [w for w in phrases if not w in stopwords]
        return removed_stopwords
    
    @staticmethod
    def clean_txt(doc):
        # Normalize txt
        new_doc = doc.lower()
        # Removing email
        new_doc = PreprocessExpert.remove_email_addr(new_doc)
        # Removing urls
        new_doc = PreprocessExpert.remove_ulrs(new_doc)
        # Removing hashtags
        new_doc = PreprocessExpert.remove_hashtags(new_doc)
        # Replace diacritics
        new_doc = PreprocessExpert.replace_ro_diacritics(new_doc)
        # Removing symbols and such
        new_doc = re.sub(r"( -)|(- )|(\d+)|([`~!@#$%^&*()_+[\]{};:'\"\\|,<.>/?]+)", "", new_doc)
        new_doc = re.sub(r"(a{2,})|(i{2,})|(o{2,})|(u{2,})|(e{3,})", "", new_doc)
        # Removing stop words - ro, en
        new_doc = PreprocessExpert.remove_stop_words_ro(new_doc)
        new_doc = PreprocessExpert.remove_stop_words_en(new_doc)
        # Stemming en
        new_doc = PreprocessExpert.stemm_en(new_doc)
        # Lemmatization ro
        new_doc = PreprocessExpert.get_lemma_ro(new_doc)
        return new_doc



In [32]:
print(PreprocessExpert.clean_txt('Aa!! castron@gi.com e fain. Peste tot si toate... message, lol. Mereu mersese!'))

fain messag merge
